In [394]:
from ortools.linear_solver import pywraplp

In [395]:
def calc_min_path (graph):
    for i in range(len(graph)):
        for j in range(len(graph)):
            for k in range(len(graph)):
                if (graph[i][j] > graph[i][k] + graph[k][j]):
                    graph[i][j] = graph[i][k] + graph[k][j]
    return graph

In [396]:
def transform_graph_in_bipartite(graph, infinity):
    #Transform the graph into bipartite graph
    graph_biapartite = []

    #Init graph_biapartite with infinity
    graph_biapartite_lengh = 2 * len(graph)

    for l in range(graph_biapartite_lengh):
        new_list = []
        for m in range(graph_biapartite_lengh):
            new_list.append(infinity)
        graph_biapartite.append(new_list)

    
    #Populate the new graph   
    for i in range(len(graph)):
        for j in range(len(graph[i])):
            graph_biapartite[i][j + len(graph)] = graph[i][j]
            graph_biapartite[i + len(graph)][j] = graph[i][j]

    return graph_biapartite

In [397]:
def add_constraints(graph, optimization_vars, optimization_constraints, max_value, solver, infinity):
    
    optimization_constraints = [0] * len(graph)
    
    for i in range(len(graph)):
        optimization_constraints[i] = solver.Constraint(0, 1)
    
    #print "init constraint: must be between 0 and 1"
    for i in range(len(optimization_vars)):
        for j in range(i , len(optimization_vars) + 1):
            if (graph[i][j] != infinity):
                #print str(graph[i][j]) + "*(" + str(i) + "-" + str(j) + ")"
                new_index_j = j - i - 1
                #print "i: " + str(i)
                #print "j: " + str(new_index_j + i + 1)
                #print optimization_vars[i][new_index_j]
                
                optimization_constraints[new_index_j + i + 1].SetCoefficient(optimization_vars[i][new_index_j], 1)
                    
                optimization_constraints[i].SetCoefficient(optimization_vars[i][new_index_j], 1)
        #print "end constraint"
    
    return optimization_constraints


In [398]:
def main():
    solver = pywraplp.Solver('SolveMatching', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

    infinity = solver.infinity()

    #Sample graph
    v = [
        [infinity, 1      , infinity , 1       , infinity], 
        [1       , infinity, 1       , 1       , 1], 
        [infinity, 1       , infinity, infinity, 1],
        [1       , 1       , infinity, infinity, 1],
        [infinity, 1       , 1       , 1       , infinity]
    ]
    
    #v = [
    #    [infinity, .2      , infinity, .75     , infinity], 
    #    [.25     , infinity, .1      , .3      , infinity], 
    #    [infinity, .1      , infinity, infinity, .1],
    #    [.25     , .1      , infinity, infinity, .1],
    #    [infinity, .1      , .1      , .1       , infinity]
    #]


    optimization_vars = []
    optimization_constraints = []
    
    #Create vars
    for i in range(len(v) - 1):
        temp_list = []
        
        for j in range(i + 1, len(v)):
            new_var = solver.NumVar(0.0, 1.0, str(i) + "-" + str(j))
            temp_list.append(new_var)   
        
        optimization_vars.append(temp_list)
        
        
    #Set objetive function
    objective = solver.Objective()
    for i in range(len(optimization_vars)):
        for j in range(i + 1, len(optimization_vars) + 1):
            if (v[i][j] != infinity):
                objective.SetCoefficient(optimization_vars[i][j - i - 1], v[i][j]) 

    objective.SetMaximization()

    #Add constraints
    optimization_constraints = add_constraints(v, optimization_vars, optimization_constraints, 1, solver, infinity)
    
    #Solver
    print "Result"
    status = solver.Solve()
    if status == solver.OPTIMAL:
        for m in range(len(optimization_vars)):
            for n in range(len(optimization_vars[m])):
                print str(optimization_vars[m][n]) + " - " +  str(optimization_vars[m][n].solution_value())

    else:  # No optimal solution was found.
        if status == solver.FEASIBLE:
            print 'Uma solucao possivel foi encontrada.'
        else:
            print 'Nao foi possivel encontrar uma solucao.'

In [399]:
main()

Result
0-1 - 0.0
0-2 - 0.0
0-3 - 1.0
0-4 - 0.0
1-2 - 0.5
1-3 - 0.0
1-4 - 0.5
2-3 - 0.0
2-4 - 0.5
3-4 - 0.0
